<a href="https://colab.research.google.com/github/samiha-mahin/Ovarian-cancer/blob/main/EfficientNetV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle tensorflow numpy pandas opencv-python matplotlib scikit-learn


In [2]:
from google.colab import files
files.upload()  # Upload your kaggle.json file


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"samihamuntahamahin","key":"1ad5aaba9143fbd38da418ad8d278398"}'}

In [3]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [4]:
!kaggle datasets list -s "ovarian cancer"

ref                                                             title                                                      size  lastUpdated                 downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  --------------------------------------------------  -----------  --------------------------  -------------  ---------  ---------------  
saurabhshahane/predict-ovarian-cancer                           Predict Ovarian Cancer                                   322564  2021-02-06 08:15:47.360000           2854         47  0.7058824        
yoshifumimiya/6-ovarian-cancer-datasets                         7 ovarian cancer datasets                             239456735  2024-04-21 13:37:49.933000           3464         48  0.7647059        
sunilthite/ovarian-cancer-classification-dataset                Ovarian Cancer Subtype Classification                3554471607  2023-10-31 08:34:17.390000           1515         38  1.0          

In [5]:
!kaggle datasets download -d bitsnpieces/ovarian-cancer-and-subtypes-dataset-histopathology --unzip


Dataset URL: https://www.kaggle.com/datasets/bitsnpieces/ovarian-cancer-and-subtypes-dataset-histopathology
License(s): CC-BY-SA-4.0


In [7]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input

# Define dataset path
DATASET_PATH = "OvarianCancer"

# Cancer classes (based on folder names)
classes = ["Mucinous", "Non_Cancerous", "Endometri", "Serous", "Clear_Cell"]
class_mapping = {class_name: i for i, class_name in enumerate(classes)}

# Image size for EfficientNetV2
IMG_SIZE = 224

# Lists to store images & labels
X, y = [], []

# Load images dynamically
for class_name in classes:
    class_path = os.path.join(DATASET_PATH, class_name)

    if not os.path.exists(class_path):
        continue  # Skip if folder doesn't exist

    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)

        img = cv2.imread(img_path)
        if img is None:
            continue  # Skip unreadable images

        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = preprocess_input(img)  # EfficientNetV2 preprocessing
        X.append(img)
        y.append(class_mapping[class_name])  # Assign label

# Convert to NumPy arrays
X = np.array(X)
y = to_categorical(y, num_classes=len(classes))  # One-hot encode labels

# Split into train & test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Dataset loaded: {len(X)} images")


Dataset loaded: 497 images


In [8]:
from tensorflow.keras.applications import EfficientNetV2S
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization

# Load pre-trained EfficientNetV2 (S = small version)
base_model = EfficientNetV2S(weights="imagenet", include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Unfreeze last 5 layers for fine-tuning
for layer in base_model.layers[-5:]:
    layer.trainable = True

# Add custom layers
x = GlobalAveragePooling2D()(base_model.output)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # Prevent overfitting
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
output_layer = Dense(len(classes), activation="softmax")(x)

# Create model
model = Model(inputs=base_model.input, outputs=output_layer)

# Compile model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


82420632/82420632 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

train_generator = datagen.flow(X_train, y_train, batch_size=16)

# Train the model
history = model.fit(train_generator, validation_data=(X_test, y_test), epochs=20)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 434s 12s/step - accuracy: 0.3406 - loss: 2.7656 - val_accuracy: 0.6000 - val_loss: 1.0893
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 295s 12s/step - accuracy: 0.5860 - loss: 1.4935 - val_accuracy: 0.5900 - val_loss: 1.2247
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 319s 12s/step - accuracy: 0.6529 - loss: 1.5233 - val_accuracy: 0.6600 - val_loss: 1.0390
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 291s 12s/step - accuracy: 0.7204 - loss: 1.0700 - val_accuracy: 0.6700 - val_loss: 2.0854
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 287s 11s/step - accuracy: 0.7223 - loss: 1.0859 - val_accuracy: 0.6700 - val_loss: 1.5305
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 290s 12s/step - accuracy: 0.7302 - loss: 0.9617 - val_accuracy: 0.5900 - val_loss: 1.5706
Epoch 7/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 292s 12s/step - accuracy: 0.6822 - loss: 1.2083 - val_accuracy: 0.6100 - val_loss: 2.7694
Epoch 8/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 286s 11s/step - accuracy: 0.7254 - loss: 1.4048 - val_accuracy: 0.